# Testing False Positive Rate control

In this notebook we test the FPR control that results from applying the bootstrap method.

In [7]:
import pyrft as pr
import sanssouci as ss
import numpy as np
from sklearn.utils import check_random_state
rng = check_random_state(101)

## FWER control

In [ ]:
# %% Multiple contrasts - global null is false
alpha = 0.1; niters = 1000;
Dim = (10,10); N = 30; 
categ = rng.choice(3, N, replace = True)
X = pr.groupX(categ); 
C = np.array([1,-1,0]); 
B = 100

number_of_false_positives = 0
store_origs = np.zeros((1,niters))
for I in np.arange(niters):
    print(I)
    lat_data = pr.wfield(Dim,N)
    minPperm, orig_pvalues, pivotal_stats, _ = pr.boot_contrasts(lat_data, X, C, B)
    alpha_quantile = np.quantile(minPperm, alpha)
    store_origs[0,I] = minPperm[0]
    if minPperm[0] < alpha_quantile:
        number_of_false_positives = number_of_false_positives + 1
        
FPR = number_of_false_positives/niters

## JER control

In [ ]:
This script checks the JER control (but can take a few minutes)

In [24]:
# %% Testing the JER control (global null)
niters = 1000
alpha = 0.1
B = 100

Dim = (20,20); N = 30; 
C = np.array([[1,-1,0],[0,1,-1]]); 

m = np.prod(Dim)

nbelow = 0 # Initialize the FPR counter

for I in np.arange(niters):
    # Keep track of the progress.
    pr.modul(I,100)
    
    categ = rng.choice(3, N, replace = True)
    X = pr.groupX(categ); lat_data = pr.wfield(Dim,N)
    # If you want to test it when you add signal!
    # w0=np.where(categ==0)
    # lat_data.field[:,:,w0] = lat_data.field[:, :, w0] + signal
    
    minPperm, orig_pvalues, pivotal_stats, _ = pr.boot_contrasts(lat_data, X, C, B)
    lambda_quant = np.quantile(pivotal_stats, alpha)

    if pivotal_stats[0] < lambda_quant:
        nbelow = nbelow + 1

# Calculate the false positive rate
FPR = nbelow/niters

# Calculate the standard error
std_error = 1.96*np.sqrt(FPR*(1-FPR)/niters)

# Print the result
print('FPR: ', FPR, ' +/- ', round(std_error,4))

0
100
200
300
400
500
600
700
800
900
FPR:  0.121  +/-  0.0202


FPR:  0.113  +/-  0.0196


In [17]:
FPR

0.113